In [1]:
#importing the read_sbml_model function from cobra
from cobra.io import read_sbml_model
#make sure model name is up to date!
model = read_sbml_model('iCW773_test7_psilocybin.xml') #assigning our model to the variable model

No objective coefficients in model. Unclear what should be optimized


In [2]:
model.optimize().objective_value

0.0

In [3]:
# no objective function is set in the published model iCW773, thus optimization eturns 0
#setting the objective to the biomass reaction of Corynebacterium glutamicum
#find the biomass function:
model.reactions.query("biomass", "id")

[<Reaction CG_biomass_cgl_ATCC13032 at 0x27d0e0de2b0>,
 <Reaction EX_biomass at 0x27d0e43e3d0>]

In [4]:
#CG_biomass_cgl_ATCC13032 is the id of the biomass reaction 
#setting it as objective
model.objective = model.reactions.CG_biomass_cgl_ATCC13032

#optimze the model for biomass production: 
model.optimize().objective_value

0.42892966213489403

In [5]:
# now we get a value and can start using the model.

In [6]:
model.reactions.CG_biomass_cgl_ATCC13032.lower_bound = 0.1

In [7]:
#model.objective = model.reactions.DM_psi_c
#(model.solve().f * 4) / (model.reactions.EX_glc_e.flux * 6)

In [8]:
from cameo import models

In [9]:
#testing something from https://github.com/biosustain/cameo-notebooks/blob/master/01-quick-start.ipynb

In [10]:
from cameo import load_model

In [11]:
model = load_model("iCW773_test7_psilocybin.xml")

No objective coefficients in model. Unclear what should be optimized


In [12]:
model.objective = model.reactions.CG_biomass_cgl_ATCC13032

In [13]:
model

Name,iCW773
Memory address,0x027d105bf4f0
Number of metabolites,955
Number of reactions,1221
Number of groups,0
Objective expression,1.0*CG_biomass_cgl_ATCC13032 - 1.0*CG_biomass_cgl_ATCC13032_reverse_81110
Compartments,"c, e"


In [14]:
solution = model.optimize()

In [15]:
solution


,fluxes,reduced_costs
ALAR,0.075971,0.000000e+00
ALATA_L,0.389078,0.000000e+00
ASNN,0.000000,-7.588265e-17
ASNS1,0.000000,-5.887306e-17
ASNS2,0.091031,-1.972152e-31
...,...,...
DM_nbc_c,0.000000,-4.464873e-15
DM_psi_c,0.000000,-8.863463e-17
DM_amet_c,0.000000,-3.122296e-15
DM_ahcys_c,0.000000,-5.310415e-15


In [16]:
# how many reactions are essential?
from cobra.flux_analysis import find_essential_reactions
print("This model for Corynebacterium glutamicum has ", len(find_essential_reactions(model)), " essential reactions of totally ", len(model.reactions), "reactions.") 

This model for Corynebacterium glutamicum has  253  essential reactions of totally  1221 reactions.


In [17]:
#how many genes are essential?
from cobra.flux_analysis import find_essential_genes
print (len(find_essential_genes(model)), "genes out of ", len(model.genes), " are essential")

178 genes out of  773  are essential


# Testing cofactor swapping again

In [18]:
#model_orig = load_model("iCW773_test7_psilocybin.xml")

In [19]:
#model = model_orig.copy()
#model.objective = model.reactions.CG_biomass_cgl_ATCC13032

In [20]:
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.objective_functions import biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva

In [21]:
model.reactions.CG_biomass_cgl_ATCC13032.lower_bound = 0.1

In [22]:
#model.objective = model.reactions.DM_psi_c
#(model.solve().f * 4) / (model.reactions.EX_glc_e.flux * 6)

In [23]:
model.objective = model.reactions.DM_psi_c

In [24]:
py = product_yield(model.reactions.DM_psi_c, model.reactions.EX_glc_e)
optswap = CofactorSwapOptimization(model=model, objective_function=py, cofactor_id_swaps=(['nadp_c', 'nadph_c'], ['nad_c', 'nadh_c']))

In [25]:
optswap.run(max_evaluations=2000, max_size=2)

Starting optimization at Fri, 19 Nov 2021 15:16:19


HBox()

Finished after 00:00:38


,index,targets,fitness


In [26]:
#Cofactor-swapping seems not to be an advantage for any reaction/enzyme, when producing psilocybin.

In [27]:
## WHich C-sources can be used by our C. glutamicum model? 

In [28]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

In [29]:
solution = model.optimize()


In [30]:
reaction_with_largest_flux = model.reactions[solution.fluxes.argmax()].id

In [31]:
#creating a list (ex_metabolites) with all metabolites associated with the compartment e by searching through all metabolite "id" with the ending "_e"
ex_metabolites = []
for metabolite in model.metabolites.query('_e', 'id'):
    ex_metabolites.append(metabolite.id)
#ex_metabolites

#creating a list (imported_c_source) by checking the metabolites of ex_metabolites for the presence of "C" --> Just the metabolites which have a "C" are added to imported_c_source
imported_c_source = []
for ex in ex_metabolites:
    if "C" in model.metabolites.get_by_id(ex).elements: 
        imported_c_source.append(ex)
#imported_c_source

#all c-sources mentioned in the list imported_c_sources are used to find the respective uptake reaction. The ids of the respective reactions are saved to the list ex_reactions_c
ex_reactions_c = []
for imp in imported_c_source:
    for reaction in model.reactions.query(imp, 'id'):
        ex_reactions_c.append(reaction.id)
ex_reactions_c

['EX_26dap_M_e',
 'EX_3amp_e',
 'EX_3cmp_e',
 'EX_3gmp_e',
 'EX_3ump_e',
 'EX_4abut_e',
 'EX_ac_e',
 'EX_nac_e',
 'EX_acald_e',
 'EX_acgam_e',
 'EX_uacgam_e',
 'EX_acgam1p_e',
 'EX_acmana_e',
 'EX_acser_e',
 'EX_ade_e',
 'EX_adn_e',
 'EX_akg_e',
 'EX_alaala_e',
 'EX_ala_B_e',
 'EX_ala_D_e',
 'EX_ala_L_e',
 'EX_3amp_e',
 'EX_amp_e',
 'EX_damp_e',
 'EX_arg_L_e',
 'EX_asn_L_e',
 'EX_asp_L_e',
 'EX_cgly_e',
 'EX_cit_e',
 'EX_3cmp_e',
 'EX_cmp_e',
 'EX_dcmp_e',
 'EX_co2_e',
 'EX_cyan_e',
 'EX_cys_L_e',
 'EX_cytd_e',
 'EX_daD_2_e',
 'EX_damp_e',
 'EX_dcmp_e',
 'EX_dcyt_e',
 'EX_dgmp_e',
 'EX_dgsn_e',
 'EX_dha_e',
 'EX_dimp_e',
 'EX_din_e',
 'EX_dms_e',
 'EX_dmso_e',
 'EX_dtmp_e',
 'EX_dump_e',
 'EX_duri_e',
 'EX_etha_e',
 'EX_etoh_e',
 'EX_f6p_e',
 'EX_for_e',
 'EX_fru_e',
 'EX_fruur_e',
 'EX_fum_e',
 'EX_g1p_e',
 'EX_g3pe_e',
 'EX_g3pg_e',
 'EX_g3pi_e',
 'EX_g3ps_e',
 'EX_g6p_e',
 'EX_gal1p_e',
 'EX_acgam_e',
 'EX_gam_e',
 'EX_uacgam_e',
 'EX_gam6p_e',
 'EX_glcn_e',
 'EX_glcur_e',
 'EX_udpg

In [32]:
# to lists also includes exported components like amino-acids
model.reactions.EX_ile_L_e

Reaction identifier,EX_ile_L_e
Name,L_Isoleucine exchange
Memory address,0x027d108b8460
Stoichiometry,ile_L_e --> L_Isoleucine -->
GPR,
Lower bound,0.0
Upper bound,2.30669833333333


In [33]:
model.reactions.EX_glc_e

Reaction identifier,EX_glc_e
Name,D_Glucose exchange
Memory address,0x027d108a8be0
Stoichiometry,glc_D_e <-- D_Glucose <--
GPR,
Lower bound,-4.67
Upper bound,-4.09906


In [34]:
c_sources = []
for c in ex_reactions_c: 
    if model.reactions.get_by_id(c).lower_bound <= 0: 
        c_sources.append(c)
        
        
c_sources

['EX_26dap_M_e',
 'EX_3amp_e',
 'EX_3cmp_e',
 'EX_3gmp_e',
 'EX_3ump_e',
 'EX_4abut_e',
 'EX_ac_e',
 'EX_nac_e',
 'EX_acald_e',
 'EX_acgam_e',
 'EX_uacgam_e',
 'EX_acgam1p_e',
 'EX_acmana_e',
 'EX_acser_e',
 'EX_ade_e',
 'EX_adn_e',
 'EX_akg_e',
 'EX_alaala_e',
 'EX_ala_B_e',
 'EX_ala_D_e',
 'EX_ala_L_e',
 'EX_3amp_e',
 'EX_amp_e',
 'EX_damp_e',
 'EX_arg_L_e',
 'EX_asn_L_e',
 'EX_asp_L_e',
 'EX_cgly_e',
 'EX_cit_e',
 'EX_3cmp_e',
 'EX_cmp_e',
 'EX_dcmp_e',
 'EX_cyan_e',
 'EX_cys_L_e',
 'EX_cytd_e',
 'EX_daD_2_e',
 'EX_damp_e',
 'EX_dcmp_e',
 'EX_dcyt_e',
 'EX_dgmp_e',
 'EX_dgsn_e',
 'EX_dha_e',
 'EX_dimp_e',
 'EX_din_e',
 'EX_dms_e',
 'EX_dmso_e',
 'EX_dtmp_e',
 'EX_dump_e',
 'EX_duri_e',
 'EX_etha_e',
 'EX_etoh_e',
 'EX_f6p_e',
 'EX_for_e',
 'EX_fru_e',
 'EX_fruur_e',
 'EX_fum_e',
 'EX_g1p_e',
 'EX_g3pe_e',
 'EX_g3pg_e',
 'EX_g3pi_e',
 'EX_g3ps_e',
 'EX_g6p_e',
 'EX_gal1p_e',
 'EX_acgam_e',
 'EX_gam_e',
 'EX_uacgam_e',
 'EX_gam6p_e',
 'EX_glcn_e',
 'EX_glcur_e',
 'EX_udpglcur_e',
 'EX

In [35]:
## id we could receive the information for import or export we could distinguish between them and identify c-sources via uptake

## Searching for alternative heterologous pathways for psilocybin production

In [36]:
## Testing alternative pathway for psilocybin production found here: https://github.com/biosustain/cameo-notebooks/blob/master/07-predict-heterologous-pathways.ipynb

In [37]:
from IPython.display import display
import re

In [38]:
from cameo import models
from cameo.strain_design import pathway_prediction

In [39]:
predictor = pathway_prediction.PathwayPredictor(model)

In [40]:
pathways = predictor.run(product="psilocybin", max_predictions=1)

ValueError: Specified product 'psilocybin' could not be found. Try searching pathway_predictor_obj.universal_model.metabolites

Psilocybin could not be found within the metabolites of the universal_model. Thus we will implement the pathway starting from L-tryptophan as published by Milne et al., 2020

In [ ]:
pathways = predictor.run(product="psilocin", max_predictions=1)

However, psilocin, was found within the universal model and could be produced by alternative pathways. This will n ot further be investigated, since this project focuses on psilocybin as product. 

In [41]:
model.optimize().objective_value

4.365741867082849e-05

In [42]:
print(model.objective.expression)
print(model.objective.direction)


1.0*DM_psi_c - 1.0*DM_psi_c_reverse_8fa95
max


## Testing different c-sources 

In [43]:
model = read_sbml_model('iCW773_test7_psilocybin.xml')
model.objective = model.reactions.CG_biomass_cgl_ATCC13032


No objective coefficients in model. Unclear what should be optimized


In [44]:

model.reactions.EX_glc_e.upper_bound


-4.09906

In [ ]:
#Changing the boundaries within the model

#model.reactions.EX_glc_e.upper_bound = 1000
#model.reactions.EX_glc_e.lower_bound = -10

#model.reactions.EX_xyL_D_e.upper_bound = 1000
#model.reactions.EX_xyL_D_e.lower_bound = -10

#model.reactions.EX_o2_e.upper_bound = 1000
#model.reactions.EX_o2_e.lower_bound = -10

In [45]:
# What is the current medium? 
model.medium

{'EX_ca2_e': 0.0144877134286247,
 'EX_cobalt2_e': 7.00338234344926e-05,
 'EX_cu2_e': 0.00197271722237563,
 'EX_fe2_e': 0.16239125,
 'EX_fe3_e': 0.0217300637039616,
 'EX_glc_e': 4.67,
 'EX_k_e': 0.543285250115532,
 'EX_mg2_e': 0.0241459221505149,
 'EX_mn2_e': 0.00192443552016793,
 'EX_nh4_e': 11.9121416666662,
 'EX_ni2_e': 0.00089824250292982,
 'EX_o2_e': 16.5067783333334,
 'EX_pi_e': 0.918000000000888,
 'EX_so4_e': 4.01673156249995,
 'EX_zn2_e': 0.000948926279874219,
 'EX_BIOTIN': 1.01925397503027e-06,
 'EX_cl': 0.0144877134286247}

In [46]:
medium = model.medium
with model:
    medium['EX_glc_e'] = 100
    medium['EX_o2_e'] = 100
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['CG_biomass_cgl_ATCC13032'])
    print(medium)

0.4289296621348931
{'EX_ca2_e': 0.0144877134286247, 'EX_cobalt2_e': 7.00338234344926e-05, 'EX_cu2_e': 0.00197271722237563, 'EX_fe2_e': 0.16239125, 'EX_fe3_e': 0.0217300637039616, 'EX_glc_e': 100, 'EX_k_e': 0.543285250115532, 'EX_mg2_e': 0.0241459221505149, 'EX_mn2_e': 0.00192443552016793, 'EX_nh4_e': 11.9121416666662, 'EX_ni2_e': 0.00089824250292982, 'EX_o2_e': 100, 'EX_pi_e': 0.918000000000888, 'EX_so4_e': 4.01673156249995, 'EX_zn2_e': 0.000948926279874219, 'EX_BIOTIN': 1.01925397503027e-06, 'EX_cl': 0.0144877134286247}


In [47]:
medium = model.medium
with model:
    medium['EX_glc_e'] = 100
    medium['EX_o2_e'] = 100
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['CG_biomass_cgl_ATCC13032'])
    print(medium)

0.4289296621348931
{'EX_ca2_e': 0.0144877134286247, 'EX_cobalt2_e': 7.00338234344926e-05, 'EX_cu2_e': 0.00197271722237563, 'EX_fe2_e': 0.16239125, 'EX_fe3_e': 0.0217300637039616, 'EX_glc_e': 100, 'EX_k_e': 0.543285250115532, 'EX_mg2_e': 0.0241459221505149, 'EX_mn2_e': 0.00192443552016793, 'EX_nh4_e': 11.9121416666662, 'EX_ni2_e': 0.00089824250292982, 'EX_o2_e': 100, 'EX_pi_e': 0.918000000000888, 'EX_so4_e': 4.01673156249995, 'EX_zn2_e': 0.000948926279874219, 'EX_BIOTIN': 1.01925397503027e-06, 'EX_cl': 0.0144877134286247}


In [49]:
medium = model.medium
model.objective = model.reactions.DM_psi_c
with model:
    medium['EX_glc_e'] = 100
    medium['EX_o2_e'] = 100
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['DM_psi_c'])
    print(medium)

5.692998837082868e-05
{'EX_ca2_e': 0.0144877134286247, 'EX_cobalt2_e': 7.00338234344926e-05, 'EX_cu2_e': 0.00197271722237563, 'EX_fe2_e': 0.16239125, 'EX_fe3_e': 0.0217300637039616, 'EX_glc_e': 100, 'EX_k_e': 0.543285250115532, 'EX_mg2_e': 0.0241459221505149, 'EX_mn2_e': 0.00192443552016793, 'EX_nh4_e': 11.9121416666662, 'EX_ni2_e': 0.00089824250292982, 'EX_o2_e': 100, 'EX_pi_e': 0.918000000000888, 'EX_so4_e': 4.01673156249995, 'EX_zn2_e': 0.000948926279874219, 'EX_BIOTIN': 1.01925397503027e-06, 'EX_cl': 0.0144877134286247}


In [69]:
medium = model.medium
model.objective = model.reactions.DM_psi_c
with model:
    medium['EX_glc_e'] = 100
    medium['EX_o2_e'] = 1.44076799999993
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['DM_psi_c'])
    print(medium)

0.0
{'EX_ca2_e': 0.0144877134286247, 'EX_cobalt2_e': 7.00338234344926e-05, 'EX_cu2_e': 0.00197271722237563, 'EX_fe2_e': 0.16239125, 'EX_fe3_e': 0.0217300637039616, 'EX_glc_e': 100, 'EX_k_e': 0.543285250115532, 'EX_mg2_e': 0.0241459221505149, 'EX_mn2_e': 0.00192443552016793, 'EX_nh4_e': 11.9121416666662, 'EX_ni2_e': 0.00089824250292982, 'EX_o2_e': 1.44076799999993, 'EX_pi_e': 0.918000000000888, 'EX_so4_e': 4.01673156249995, 'EX_zn2_e': 0.000948926279874219, 'EX_BIOTIN': 1.01925397503027e-06, 'EX_cl': 0.0144877134286247}


In [56]:
model.metabolites.query("met", "name")

[<Metabolite 2dmmq8_c at 0x27d1a801af0>,
 <Metabolite 2dmmql8_c at 0x27d124c3070>,
 <Metabolite 2mahmp_c at 0x27d125dd460>,
 <Metabolite 4ampm_c at 0x27d12052bb0>,
 <Metabolite 5prdmbz_c at 0x27d10794c40>,
 <Metabolite 6hmhpt_c at 0x27d107942b0>,
 <Metabolite 6hmhptpp_c at 0x27d0e575670>,
 <Metabolite amet_c at 0x27d1066f970>,
 <Metabolite ametam_c at 0x27d106cb100>,
 <Metabolite dmbzid_c at 0x27d1bf060a0>,
 <Metabolite dmlz_c at 0x27d19ccee80>,
 <Metabolite dmpp_c at 0x27d19cced30>,
 <Metabolite dms_c at 0x27d19ccefd0>,
 <Metabolite dms_e at 0x27d19cce130>,
 <Metabolite dmso_c at 0x27d19cce6a0>,
 <Metabolite dmso_e at 0x27d1bf06490>,
 <Metabolite fmettrna_c at 0x27d19c98670>,
 <Metabolite hmbil_c at 0x27d1becbac0>,
 <Metabolite micit_c at 0x27d19cecfd0>,
 <Metabolite prfp_c at 0x27d1bf1a6a0>,
 <Metabolite prlp_c at 0x27d1bf1a4c0>,
 <Metabolite tma_e at 0x27d1be96e50>,
 <Metabolite tmao_e at 0x27d19cf1070>]

In [ ]:
model.reactions.EX_o2_e

In [ ]:
model.metabolites.query("o2", "id")

In [77]:
model.metabolites.met_L_e

Metabolite identifier,met_L_e
Name,L_Methionine
Memory address,0x027d19cece80
Formula,C5H11NO2S
Compartment,e
In 3 reaction(s),"METabcpp, EX_met_L_e, METt2rpp"


In [74]:
model.metabolites.amet_c

Metabolite identifier,amet_c
Name,S-AdenosyL_L_methionine
Memory address,0x027d1066f970
Formula,C15H23N6O5S
Compartment,c
In 13 reaction(s),"UPP3MT, BTS4, AMAOTr, OMBZLM, DM_amet_c, SOL_POOL, TYRL, CPPPGO2, METAT, AMMQLT8, psiM, psiM2, LIPOS"


In [76]:
model.reactions.METAT # reaction producing amet_c from L_Methionine

Reaction identifier,METAT
Name,methionine adenosyltransferase
Memory address,0x027d1b8bbf40
Stoichiometry,atp_c + h2o_c + met_L_c --> amet_c + pi_c + ppi_c ATP + H2O + L_Methionine --> S-AdenosyL_L_methionine + Phosphate + Diphosphate
GPR,cg1806
Lower bound,0.0
Upper bound,0.000113859976741657


In [79]:
model.reactions.EX_met_L_e

Reaction identifier,EX_met_L_e
Name,L_Methionine exchange
Memory address,0x027d12426910
Stoichiometry,met_L_e --> L_Methionine -->
GPR,
Lower bound,0.0
Upper bound,0.537786666666667


In [103]:
# Adding L-methionin, which is the precursor of SAM (Cofactors used for psilocybin production) to the medium
medium = model.medium
model.objective = model.reactions.DM_psi_c
with model:
    medium['EX_glc_e'] = 100
    medium['EX_o2_e'] = 1.5
    medium["EX_met_L_e"] = 100
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['DM_psi_c'])

0.0


C:\Users\tamwei\Anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [97]:
with model:
    model.objective = model.reactions.psiK2
    print(model.optimize().objective_value)

5.57043656605488


In [98]:
with model:
    model.objective = model.reactions.psiM2
    print(model.optimize().objective_value)

5.692998837082857e-05
